In [ ]:

def print_table(t, h):
    from tabulate import tabulate
    print(tabulate(t, headers=h, tablefmt='orgtbl'))

print_table([[1,2],[1,2]], ['1', '2'])

|   1 |   2 |
|-----+-----|
|   1 |   2 |
|   1 |   2 |


In [ ]:
import random

def honestpaper(I,dont,care,about_these):
    return "P"

def honestrock(I,dont,care,about_these):
    return "R"

def honestscissors(I,dont,care,about_these):
    return "S"

def honestcircle(other_past, my_past, other_next, my_next):
    winners = {"R": "P", "P": "S", "S": "R"}
    return list(winners.keys())[len(my_past)%3]

def test_bot(other_past, my_past, other_next, my_next):
    winners = {"R": "P", "P": "S", "S": "R"}
    if my_next is None:
        if len(other_past) != 0:
            if len(other_past) > 2 and random.random() < 0.3:
                return winners[other_past[-2][1]]
            else:
                return winners[other_past[-1][1]]
        else:
            return random.choice(["R","P","S"])
    return random.choice(["R","P","S"]) if random.random() < 0.1 else my_next


def test_bot2(other_past, my_past, other_next, my_next):
    winners = {"R": "P", "P": "S", "S": "R"}
    if my_next is None:
        if len(other_past) != 0:
            return winners[other_past[-1][0]]
        else:
            return random.choice(["R","P","S"])
    return my_next


def randombot(other_past, my_past, other_next, my_next):
    return random.choice(["R","P","S"])

def honestrandom(a, b, c, move):
    if move == None:
        return random.choice(["R","P","S"])
    return move

def adaptive_bot(other_past, my_past, other_next, my_next):
    winners = {"R": "P", "P": "S", "S": "R"}
    if my_next is None:
        return winners[other_past[-6:][0][1]] if len(other_past) > 0 else random.choice(list(winners.keys()))
    else:
        is_other_honest = all([other_claim == other_move for other_claim, other_move in other_past[-2:]])
        return winners[other_next] if is_other_honest else my_next

def tit4tat(opphist, myhist, oppfut, myfut):
    if (not myfut): return random.choice(['R','P','S'])
    if (not opphist) or opphist[-1][0]==opphist[-1][1]: return myfut
    return random.choice(['R','P','S'])


def csbot(ophist,myhist,opdecl,mydecl):

  import random

  RPS = "RPS"

  def value(opd,myd,opmove,mymove):
    if opmove==mymove:
      val = 9
    elif opmove+mymove in RPS+RPS:
      val = 20
    else:
      val = -2
    return val+10*(myd==mymove)-(opd==opmove)

  def best(od,md):
    l = float(len(ophist))
    weights = dict([ (m, random.random()/8) for m in RPS ])
    for n in range(len(ophist)):
      if ophist[n][0]==od and myhist[n][0]==md:
        weights[ophist[n][1]] += 1+4*((n+1)/l)**2
    sw = sum([ weights[m] for m in RPS ])
    bestexpect = 0
    for m in RPS:
      expect = sum([ weights[om]/sw*value(od,md,om,m) for om in RPS ])
      if expect > bestexpect:
        bestexpect = expect
        bestmove = m
    return bestmove, bestexpect


  honest = all ([ decl==mv for decl, mv in ophist ])

  if honest:
    if not (mydecl is None):
      return mydecl
    expnxt = set();
    for i in range(len(ophist)-1):
      if ophist[i][0]==ophist[-1][0]:
        expnxt.add(ophist[i+1][0])
    if len(expnxt)==1:
      return RPS[ (RPS.index(expnxt.pop())+1) % 3 ]

  if mydecl==None:
    l = float(len(ophist))
    weights = dict([ (m, random.random()) for m in RPS ])
    for n in range(len(ophist)):
      weights[ophist[n][0]] += 1+((n+1)/l)**2
    sw = sum([ weights[m] for m in RPS ])
    bestexpect = 0
    worstexpect = 99
    for m in RPS:
      expect = sum([ best(od,m)[1]/sw*weights[od] for od in RPS ])
      if expect > bestexpect:
        bestexpect = expect
        bestmove = m
      if expect < worstexpect:
        worstexpect = expect
    if bestexpect-worstexpect < 3:
      bestmove = random.choice(RPS)
    return bestmove

  return best(opdecl,mydecl)[0]

In [ ]:
bot_map = {
    0:csbot,
    1:honestpaper,
    2:honestscissors,
    3:randombot,
    4:honestrandom,
    5:adaptive_bot,
    6:tit4tat,
    7:test_bot,
    8:test_bot2,
    9:honestcircle,
}

player_num=len(bot_map)

def real(history1,history2,number,honest1,honest2):
    return bot_map[number](history1,history2,honest1,honest2)

def honest(history1,history2,number):
    return bot_map[number](history1,history2,None,None)

def play_match(num1,num2):
    history1=[]
    history2=[]
    score1=0
    score2=0
    for x in range(250):
        h1=honest(history2,history1,num1)
        h2=honest(history1,history2,num2)

        r1=real(history2,history1,num1,h2,h1)
        r2=real(history1,history2,num2,h1,h2)

        if h1==r1: score1 += 1
        if h2==r2: score2 += 1

        if r1==r2: score1+=1; score2+=1
        elif r1=="R":
            if r2=="P": score2+=2
            else: score1+=2
        elif r1=="P":
            if r2=="S": score2+=2
            else: score1+=2
        else:
            if r2=="R": score2+=2
            else: score1+=2

        if h1 is None or r1 is None: print(num1,num2)
        if h2 is None or r2 is None: print(num1,num2)

        history1.append([h1,r1])
        history2.append([h2,r2])
    return score1,score2

clamp = lambda t : max(min(t, 20), -20)
from math import floor
from tqdm import tqdm 

ratings = [1 ]*player_num

K = 1

scores = []

for x in range(player_num):
    scores.append(0)

for _ in tqdm(range(200)):
    for x in range(player_num):
        for y in range(player_num):
            scorex,scorey=play_match(x,y)
            
            scores[x]+=scorex
            scores[y]+=scorey

            # Ex = 1/(1 + 10**(  clamp((ratings[y] - ratings[x])/400 ))  )
            # Ey = 1/(1 + 10**(  clamp((ratings[x] - ratings[y])/400 ))  )

            # ratings[x] = ratings[x] + K*(scorex - Ex)
            # ratings[y] = ratings[y] + K*(scorey - Ey)

out = []

for k, score in enumerate(scores):
    #print(bot_map[k].__name__, score )
    out.append((bot_map[k].__name__, floor(ratings[k]), score ))

print('')

out = sorted(out, key = lambda t: t[2], reverse = True)
print_table(out, ['bot name', 'rating', 'score'])

100%|██████████| 200/200 [04:17<00:00,  1.29s/it]


| bot name       |   rating |   score |
|----------------+----------+---------|
| csbot          |        1 | 2323532 |
| adaptive_bot   |        1 | 2205415 |
| test_bot2      |        1 | 2114223 |
| test_bot       |        1 | 1923900 |
| honestrandom   |        1 | 1899783 |
| tit4tat        |        1 | 1894932 |
| honestcircle   |        1 | 1827778 |
| honestscissors |        1 | 1710783 |
| honestpaper    |        1 | 1512386 |
| randombot      |        1 | 1333523 |


In [ ]:
from random import randint, random


def randombot(my_history, my_resources, my_state = 0):
    placed_s = [i[1] for i in my_history if i[0] != 'H']
        
    targets = ((0, 1, 2), (3, 4, 5), (5, 6, 7),
                (0, 3, 5), (1, 4, 6), (2, 5, 7),
                (0, 4, 7), (2, 4, 5))

    if len(my_history) == 0:
        return ('S', randint(0, 8))
    else:
        if my_resources[1] <= 2 and random()<0.5: 
            max_target = max(targets, key = lambda t: (t[0] in placed_s)+(t[1] in placed_s)+(t[2] in placed_s))
            return ('C', targets.index(max_target)) 

        if my_resources[1] != 0:
            return ('S', randint(0, 8))
        if my_resources[0] != 0:
            return ('B', randint(0, 8))
    
    return ('C', randint(0, 7))

def randombot_wait(my_history, my_resources, my_state = 0):
    placed_s = [i[1] for i in my_history if i[0] != 'H']
        
    targets = ((0, 1, 2), (3, 4, 5), (5, 6, 7),
                (0, 3, 5), (1, 4, 6), (2, 5, 7),
                (0, 4, 7), (2, 4, 5))

    sum_in = lambda t: (t[0] in placed_s)+(t[1] in placed_s)+(t[2] in placed_s)

    if len(my_history) == 0:
        return ('S', randint(0, 8))
    else:
        if my_resources[1] <= 1: 
            max_target = max(targets, key = sum_in)
            if sum_in(max_target) == 3:return ('C', targets.index(max_target)) 

        if my_resources[1] != 0:
            return ('S', randint(0, 8))
        if my_resources[0] != 0:
            return ('B', randint(0, 8))
    
    return ('C', randint(0, 7))

def randombot_wait_longer(my_history, my_resources, my_state = 0):
    placed_s = [i[1] for i in my_history if i[0] != 'H']
        
    targets = ((0, 1, 2), (3, 4, 5), (5, 6, 7),
                (0, 3, 5), (1, 4, 6), (2, 5, 7),
                (0, 4, 7), (2, 4, 5))
    
    sum_in = lambda t: (t[0] in placed_s)+(t[1] in placed_s)+(t[2] in placed_s)

    if len(my_history) == 0:
        return ('S', randint(0, 8))
    else:
        if my_resources[1] == 0: 
            max_target = max(targets, key = sum_in)
            if sum_in(max_target) == 3:return ('C', targets.index(max_target)) 

        if my_resources[1] != 0:
            return ('S', randint(0, 8))
        if my_resources[0] != 0:
            return ('B', randint(0, 8))

    return ('C', randint(0, 7))

def randombot_claim_first(my_history, my_resources, my_state = 0):
    placed_s = [i[1] for i in my_history if i[0] != 'H']
        
    targets = ((0, 1, 2), (3, 4, 5), (5, 6, 7),
                (0, 3, 5), (1, 4, 6), (2, 5, 7),
                (0, 4, 7), (2, 4, 5))
    
    sum_in = lambda t: (t[0] in placed_s)+(t[1] in placed_s)+(t[2] in placed_s)

    if len(my_history) == 0:
        return ('S', randint(0, 8))
    else:
        max_target = max(targets, key = sum_in)
        if sum_in(max_target) == 3:return ('C', targets.index(max_target)) 

        if my_resources[1] != 0:
            return ('S', randint(0, 8))
        if my_resources[0] != 0:
            return ('B', randint(0, 8))

    return ('C', randint(0, 7))

def randombot_suspicious(my_history, my_resources, my_state = 0):
    placed_s = [i[1] for i in my_history if i[0] != 'H']
    sus_s =  [i[1] for i in my_history if i[0] == 'H']

    targets = ((0, 1, 2), (3, 4, 5), (5, 6, 7),
                (0, 3, 5), (1, 4, 6), (2, 5, 7),
                (0, 4, 7), (2, 4, 5))
    
    sum_in = lambda t: (t[0] in placed_s)+(t[1] in placed_s)+(t[2] in placed_s)
    sum_in_sus = lambda t: (t[0] in sus_s)+(t[1] in sus_s)+(t[2] in sus_s)

    if len(my_history) == 0:
        return ('S', randint(0, 8))
    else:
        max_target = max(targets, key = sum_in)
        if sum_in(max_target) == 3 and sum_in_sus(max_target) == 0:return ('C', targets.index(max_target)) 

        if my_resources[1] != 0:
            return ('S', randint(0, 8))
        if my_resources[0] != 0:
            return ('B', randint(0, 8))

    return ('C', randint(0, 7))

def randombot_baiting(my_history, my_resources, my_state = 0):
    placed_s = [i[1] for i in my_history if i[0] != 'H']
        
    targets = ((0, 1, 2), (3, 4, 5), (5, 6, 7),
                (0, 3, 5), (1, 4, 6), (2, 5, 7),
                (0, 4, 7), (2, 4, 5))
    
    sum_in = lambda t: (t[0] in placed_s)+(t[1] in placed_s)+(t[2] in placed_s)

    if len(my_history) == 0:
        return ('S', randint(0, 8))
    else:
        max_target = max(targets, key = sum_in)
        if random() < 0.1 and sum_in(max_target) == 3:return ('C', targets.index(max_target)) 

        if my_resources[1] != 0 and random()<0.5:
            return ('S', randint(0, 8))
        if my_resources[0] != 0:
            return ('B', randint(0, 8))

    return ('C', randint(0, 7))

In [ ]:
def print_table(t, h):
    from tabulate import tabulate
    print(tabulate(t, headers=h, tablefmt='orgtbl'))

bot_map = {
    0:randombot,
    1:randombot_wait,
    2:randombot_wait_longer,
    3:randombot_claim_first,
    4:randombot_suspicious,
    5:randombot_baiting,
}

# S_ymbol B_lank H_idden C_laim W_in L_ose

player_num=len(bot_map)

targets = ((0, 1, 2), (3, 4, 5), (5, 6, 7),
           (0, 3, 5), (1, 4, 6), (2, 5, 7),
           (0, 4, 7), (2, 4, 5))

def play_match_tic_tac(num1,num2):
    actual_board = [0]*9
    
    resources = [[5,5],[5,5]]
    history = [[], []]
    states = [[], []]
    nums = [num1,num2]

    score0=0
    score1=0

    for x in range(40):
        player = x%2
        other = (1+player)%2

        r = bot_map[nums[player]](history[player],resources[player])
        #print(r)

        #logic here 
        if r[0] == 'S':
            assert resources[player][1] > 0
            actual_board[r[1]] += 1 if player == 0 else -1
            resources[player][1] -= 1
        elif r[0] == 'B':
            assert resources[player][0] > 0
            resources[player][0] -= 1
        elif r[0] == 'C':
            if player == 0:
                if all(actual_board[j] > 0 for j in targets[r[1]]):
                    score0 = 1; score1 = 0
                else:
                    score0 = 0; score1 = 1
            if player == 1:
                if all(actual_board[j] < 0 for j in targets[r[1]]):
                    score0 = 0; score1 = 1
                else:
                    score0 = 1; score1 = 0
        else:
            assert False 
        #logic here 

        history[player].append(r)
        if r[0] == 'S' or r[0] == 'B':
            history[other].append(('H', r[1]))
        else:
            history[other].append(r)

        if score0 != 0 or score1 != 0:
            if score0 == 1:
                history[0].append(('W', 0))
                history[1].append(('L', 1))
            if score1 == 1:
                history[0].append(('L', 0))
                history[1].append(('W', 1))
            return score0, score1, history

    return score0, score1, history

def tournament(bot_map, iters = 200, p = 100):
    from tqdm import tqdm 

    player_num=len(bot_map)
    scores = [0]*player_num

    for _ in tqdm(range(iters)):
        for x in range(player_num):
            for y in range(player_num):
                
                for t in range(p):
                    scorex, scorey, h = play_match_tic_tac(x,y)
                    
                    scores[x] += scorex
                    scores[y] += scorey

    out = []
    for k, score in enumerate(scores):
        out.append((bot_map[k].__name__, score ))
    out = sorted(out, key = lambda t: t[1], reverse = True)

    print('')
    print_table(out, ['bot name', 'score'])


tournament(bot_map, 200)

100%|██████████| 200/200 [01:01<00:00,  3.25it/s]


| bot name              |   score |
|-----------------------+---------|
| randombot_suspicious  |  190299 |
| randombot_baiting     |  138852 |
| randombot_wait_longer |  119399 |
| randombot_wait        |  115120 |
| randombot_claim_first |  114630 |
| randombot             |   41700 |


In [ ]:
rcd = [((0, 0), (1, 0), (2, 0)),
        ((0, 1), (1, 1), (2, 1)),
        ((2, 1), (0, 2), (1, 2)),
        ((0, 0), (0, 1), (2, 1)),
        ((1, 0), (1, 1), (0, 2)),
        ((2, 0), (2, 1), (1, 2)),
        ((0, 0), (1, 1), (1, 2)),
        ((2, 0), (1, 1), (2, 1))]

# 0 1 2
# 3 4 5
# 6 7 8

sums = [[0,1,2],[3,4,5],[6,7,8],
        [0,3,6],[1,4,7],[2,5,8],
        [0,4,8],[2,4,6],]

targets = tuple([tuple(i) for i in sums])

print((targets))


((0, 1, 2), (3, 4, 5), (6, 7, 8), (0, 3, 6), (1, 4, 7), (2, 5, 8), (0, 4, 8), (2, 4, 6))


In [ ]:
print((3 > 2) + (4 > 2))

2
